Goal: Preprocess the new data and apply the model to it. Save and export the predictions.

files needed
1. new email batch
2. nickname dictionary
3. repository of functions called to this script

In [1]:
# libraries needed
import pandas as pd
import numpy as np
import time
import pickle

In [2]:
exec(open("C:/Users/jalee/Desktop/Email_Model/Function_Repository.py").read())

Alternate ways of calling the repository
1.!python C:/Users/jalee/Desktop/Email_Model/Function_Repository.py
2.%load_ext autoreload
%autoreload 1
%aimport C:/Users/jalee/Desktop/Email_Model/Function_Repository
3.import the user-defined functions
import sys
sys.path.insert(0, 'C:/Users/jalee/Desktop/Email_Model/' )
import Function_Repository

Upload the new data to be predicted: 
Before preprocessing, this data needs PPD's type of practice and telephone records

In [3]:
ppd = PPD_load()

In [16]:
data_describe(ppd)

dimension of dataframe is: (1262552, 81)
column names of dataframe are: ['ME' 'RECORD_ID' 'UPDATE_TYPE' 'ADDRESS_TYPE' 'MAILING_NAME' 'LAST_NAME'
 'FIRST_NAME' 'MIDDLE_NAME' 'SUFFIX' 'MAILING_LINE_1' 'MAILING_LINE_2'
 'CITY' 'STATE' 'ZIP' 'SECTOR' 'CARRIER_ROUTE'
 'ADDRESS_UNDELIVERABLE_FLAG' 'FIPS_COUNTY' 'FIPS_STATE'
 'PRINTER_CONTROL_CODE' 'PC_ZIP' 'PC_SECTOR' 'DELIVERY_POINT_CODE'
 'CHECK_DIGIT' 'PRINTER_CONTROL_CODE_2' 'REGION' 'DIVISION' 'GROUP'
 'TRACT' 'SUFFIX_CENSUS' 'BLOCK_GROUP' 'MSA_POPULATION_SIZE'
 'MICRO_METRO_IND' 'CBSA' 'CBSA_DIV_IND' 'MD_DO_CODE' 'BIRTH_YEAR'
 'BIRTH_CITY' 'BIRTH_STATE' 'BIRTH_COUNTRY' 'GENDER' 'TELEPHONE_NUMBER'
 'PRESUMED_DEAD_FLAG' 'FAX_NUMBER' 'TOP_CD' 'PE_CD' 'PRIM_SPEC_CD'
 'SEC_SPEC_CD' 'MPA_CD' 'PRA_RECIPIENT' 'PRA_EXP_DT' 'GME_CONF_FLG'
 'FROM_DT' 'TO_DT' 'YEAR_IN_PROGRAM' 'POST_GRADUATE_YEAR' 'GME_SPEC_1'
 'GME_SPEC_2' 'TRAINING_TYPE' 'GME_INST_STATE' 'GME_INST_ID'
 'MEDSCHOOL_STATE' 'MEDSCHOOL_ID' 'MEDSCHOOL_GRAD_YEAR' 'NO_CONTACT_IND'
 'NO

In [4]:
# import the new email batch
data = pd.read_csv('jl.VerticalTrailData.08132019.csv', converters={'ME': lambda x: str(x)})

In [5]:
# make sure this email batch contains ME numbers
data_describe(data)

dimension of dataframe is: (5404, 4)
column names of dataframe are: ['ME' 'email' 'first_name' 'last_name']
data types of columns are: ME            object
email         object
first_name    object
last_name     object
dtype: object
below is the first 5 results: 
          ME                         email first_name last_name
0  102021177          askdrkel@hotmail.com     KELLEY    TAYLOR
1  102021461              corveyr@apsu.edu      KEITH    GIPSON
2  102040597             telacyjr@tamu.edu      LACEY    THOMAS
3  102040929       nalvmacouncil@gmail.com     KELLIE  MATTHEWS
4  102041178  willsvalleymedical@yahoo.com    FRANCES       KOE


In [6]:
# join the two tables
df = pd.merge(data,ppd[['ME','TELEPHONE_NUMBER','TOP_CD']],on='ME', how='inner') 
# if you need first names and last names too, then ppd[['ME','FIRST_NAME','LAST_NAME','TELEPHONE_NUMBER','TOP_CD']]

In [7]:
# sanity check
data_describe(df)

dimension of dataframe is: (1133, 6)
column names of dataframe are: ['ME' 'email' 'first_name' 'last_name' 'TELEPHONE_NUMBER' 'TOP_CD']
data types of columns are: ME                  object
email               object
first_name          object
last_name           object
TELEPHONE_NUMBER    object
TOP_CD              object
dtype: object
below is the first 5 results: 
            ME                      email first_name last_name  \
0  10401020026         smayfield@ucsd.edu     STEVEN  MAYFIELD   
1  10401080282  DrB@TheBartlettClinic.com    HEATHER  BARTLETT   
2  10401080673        jalbert@citadel.edu   JENNIFER    ALBERT   
3  10401081238    sigmabetadelta@umsl.edu      ATHOR   MIKHAIL   
4  10401090113                 jwu@ku.edu       JUDY        WU   

  TELEPHONE_NUMBER TOP_CD  
0              NaN    020  
1       6082636420    020  
2       6038830005    020  
3              NaN    020  
4       6238821682    020  


In [8]:
# change column names
df.columns = ['ME','email','first_name','last_name','telephone','top']
df.columns.values

array(['ME', 'email', 'first_name', 'last_name', 'telephone', 'top'],
      dtype=object)

After joining the tables, preprocess the data 

In [9]:
# Change Type of Practice (top) codes into actual values
df['top'] = df['top'].apply(TOP_CD)

In [10]:
# factorize top
df['top'] = df['top'].astype('category')

In [11]:
# create a new variable: does telephone exist? (boolean)
df['telephone']= pd.isnull(df['telephone'])

In [12]:
# create a new temporary variable: email domain (factor)
df['domain'] = df['email'].apply(domain) # we still want to keep what comes after period . because of edu

In [13]:
# look at the distribution of email domain
df['domain'].value_counts(1).head(7)

gmail.com         0.128861
yahoo.com         0.037070
aol.com           0.013239
hotmail.com       0.013239
mountsinai.org    0.007944
GMAIL.COM         0.007061
bcm.edu           0.007061
Name: domain, dtype: float64

In [14]:
df['email_domain'] = np.vectorize(domain_conditions)(df['domain'], df['last_name'], df['first_name'])

In [16]:
# create an email name column
df['name'] = df['email'].apply(email_name)
df['name'].head(5)

0         smayfield
1               DrB
2           jalbert
3    sigmabetadelta
4               jwu
Name: name, dtype: object

In [17]:
df['email_name'] = np.vectorize(email_name_conditions)(df['name'], df['first_name'], df['last_name'])

In [18]:
df['email_name']=df['email_name'].fillna('no_name')

In [19]:
# creates a boolean variable for whether email name contains a number
df['NumberedEmail'] = df['name'].apply(hasNum)

In [20]:
# remove numbers and see if the last two characters are md
# creates a boolean variable for whether email name contains 'md'
df['MD'] = df['name'].apply(hasMD)

In [21]:
# the first two characters are dr
# creates a boolean variable for whether email name contains 'dr'
df['DR'] = df['name'].apply(hasDR)

In [22]:
# creates a boolean variable for whether email name contains 'doc'
df['DOC'] = df['name'].apply(hasDOC)
df.head(5)

,ME,email,first_name,last_name,telephone,top,domain,email_domain,name,email_name,NumberedEmail,MD,DR,DOC
0,10401020026,smayfield@ucsd.edu,STEVEN,MAYFIELD,True,unclassified,ucsd.edu,edu,smayfield,If_Fl,False,False,False,False
1,10401080282,DrB@TheBartlettClinic.com,HEATHER,BARTLETT,False,unclassified,TheBartlettClinic.com,etc,DrB,None,False,False,True,False
2,10401080673,jalbert@citadel.edu,JENNIFER,ALBERT,False,unclassified,citadel.edu,edu,jalbert,If_Fl,False,False,False,False
3,10401081238,sigmabetadelta@umsl.edu,ATHOR,MIKHAIL,True,unclassified,umsl.edu,edu,sigmabetadelta,no_name,False,False,False,False
4,10401090113,jwu@ku.edu,JUDY,WU,False,unclassified,ku.edu,edu,jwu,If_Fl,False,False,False,False


In [23]:
# subset only the columns needed
df2=df.iloc[:, [4,5,7,9,10,11,12,13]] # df.loc is buggy
df2.head()

,telephone,top,email_domain,email_name,NumberedEmail,MD,DR,DOC
0,True,unclassified,edu,If_Fl,False,False,False,False
1,False,unclassified,etc,None,False,False,True,False
2,False,unclassified,edu,If_Fl,False,False,False,False
3,True,unclassified,edu,no_name,False,False,False,False
4,False,unclassified,edu,If_Fl,False,False,False,False


In [24]:
# Change the data types so they are amenable to model
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for i in range(8): # 9 = the number of columns in df
    df2.iloc[:,i] = le.fit_transform(df2.iloc[:,i])
df2.head() # look at what changed to the df

C:\Users\jalee\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,telephone,top,email_domain,email_name,NumberedEmail,MD,DR,DOC
0,1,0,3,3,0,0,0,0
1,0,0,4,5,0,0,1,0
2,0,0,3,3,0,0,0,0
3,1,0,3,8,0,0,0,0
4,0,0,3,3,0,0,0,0


In [35]:
# save the prediction data
timestr = time.strftime("%m%d%Y")
user_initial = 'jl'
file_name = 'VT_Prediction_preprocessed_data'
df.to_csv('C:/Users/jalee/Desktop/' + user_initial + '.'+ file_name + '.' + timestr + '.csv', index=True)

In [37]:
# Load the LDA model
pkl_filename = "lda.pkl"
with open(pkl_filename, 'rb') as file:
    lda = pickle.load(file)

In [38]:
# predict the data
new_data_pred = lda.predict(df2)

In [39]:
# add the predictions to the original data
df['predictions'] = new_data_pred 
df.loc[df['predictions'] == 0, 'predictions'] = 'Bad'
df.loc[df['predictions'] == 1, 'predictions'] = 'Good'
df.head()

In [40]:
# export this dataframe
# make sure to adjust the user initial and file_name
timestr = time.strftime("%m%d%Y")
user_initial = 'jl'
file_name = 'VT_Predictions'
df.to_csv('C:/Users/jalee/Desktop/' + user_initial + '.'+ file_name + '.' + timestr + '.csv', index=True)